<a href="https://colab.research.google.com/github/max-ostapenko/ga4_data_import/blob/main/scripts/GCS_to_GA4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @markdown 1. Authenticate Google Colab to your GCP project
# Or run `gcloud auth application-default login` in terminal if you don't use Colab
import os

if os.getenv("COLAB_RELEASE_TAG"):
    !gcloud config set project {GCP_PROJECT_ID}
    from google.colab import auth
    auth.authenticate_user()
    !pip install -q git+https://github.com/max-ostapenko/ga4_data_import.git
else:
    %pip install -q -e ../

In [ ]:
# @markdown 2. Define the input variables
GCP_PROJECT_ID = "max-ostapenko"  # @param {type: "string"}
ZONE = "us-central1-a"  # @param {type: "string"}
INSTANCE_NAME = "sftp-server-ga4"  # @param {type: "string"}
BUCKET_NAME = "sftp-bucket-ga4"  # @param {type: "string"}
SFTP_USERNAME = "sftp-user-ga4"  # @param {type: "string"}

REGION = ZONE[:-2]


In [ ]:
# @markdown 3. Reserve static IP for your server
from ga4_data_import.compute import create_static_address

INSTANCE_IP = create_static_address(GCP_PROJECT_ID, REGION, INSTANCE_NAME)

print(
    f"""Instance IP `{INSTANCE_IP}` is reserved in your project: https://console.cloud.google.com/networking/addresses/list?project={GCP_PROJECT_ID}"""
)


In [ ]:
# @markdown 4. Create VM Instance for SFTP server and GCS bucket. Mount a bucket as read-only on SFTP server.
from ga4_data_import.compute import create_instance
from ga4_data_import.storage import create_bucket, add_bucket_read_access

instance = create_instance(
    project_id=GCP_PROJECT_ID,
    zone=ZONE,
    instance_name=INSTANCE_NAME,
    static_address=INSTANCE_IP,
    bucket_name=BUCKET_NAME,
    sftp_username=SFTP_USERNAME,
)

create_bucket(BUCKET_NAME, REGION)
add_bucket_read_access(BUCKET_NAME, instance.service_accounts[0].email)

print(
    f"""VM instance `{INSTANCE_NAME}` is available in your project: https://console.cloud.google.com/compute/instancesDetail/zones/{ZONE}/instances/{INSTANCE_NAME}?project={GCP_PROJECT_ID}"""
)
print(
    f"""Bucket `{BUCKET_NAME}` is available in your project: https://console.cloud.google.com/storage/browser/{BUCKET_NAME}?project={GCP_PROJECT_ID}"""
)

print(
    f"""\n
Now you can enter your SFTP server url to Data Import UI:
    sftp://{INSTANCE_IP}/cost_data.csv
where `cost_data.csv` - name of a file in the bucket root"""
)


In [ ]:
# @markdown 5. Paste and push Public Key from GA4 to SFTP server for authorisation
from ga4_data_import.compute import add_server_pub_key

KEY_VALUE = "ssh-rsa AAAAB3NzaC1yc2EAAAADAQABAAABgQDeDrtHfPb8TM6/BqANwQJdSmAGjygLE+2jMdIxfJZFRdcEogMLMrEARhzj9fjRQKw7RhudOpxkyvMjVRXlv6zvcbvmJhQCxk6yp3Cgq3U7ux2PYppBWSxei1R4DH2TSjy/k/7nuKDvdDaBwUC/WLOgjvQ5nKLbYp53RbWoPrv63YEDDEyuLjMww9rlJ+J9iL/fd1TxlZxvwVvPCxweWCzfQfLP6MWGvDmUMfe7eTg11rt3p/wdNUa/RxKC1HOJRWloGCw7dX5LgZYoCq8tKOS8rpHt8BoTFhKjoPsIw8h4uNsjOb9y6YSOQrQFaXj3o2l4X0ssLxkKQcaG+EDZ8PtUr/WnuPpfHCuTNN1Q6TnaF2t2zm4J7C6LchPMgiP/vO+wwloQpgg9tUVGC4FZxQhHTLmCoW+19KIuBu4nm+zgaeIWXuHfnPLE4UdFsIB3o76tAfZprrykjaWVLPc6PFVlvRx1PIAPTL7dHaajnv1rxuQdQlNU2a2dOE4KNoxnffE= Google Analytics Data Import Key"  # @param {type: "string"}
add_server_pub_key(
    GCP_PROJECT_ID,
    ZONE,
    INSTANCE_NAME,
    KEY_VALUE,
    SFTP_USERNAME,
)

print(
    f"""Public Key is added to your SFTP server. Now you can connect to your SFTP server with your private key."""
)
